In [ ]:
# from fastapi import FastAPI
# from pydantic import BaseModel

# app = FastAPI()

# class Item(BaseModel):
#     url: str

# def test(url):
#     #insert AI Model
#     return url

# @app.post("/nlpmodel/")
# async def create_item(item: Item):
#     return test(item.url)

In [1]:
from fastapi import FastAPI
from pydantic import BaseModel

In [2]:
import fitz
import io
from PIL import Image
import numpy as np
import pytesseract

In [5]:
import yake
text=''

In [3]:
import spacy
import pytextrank

d:\PYTHON\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import PyPDF2

In [7]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Tesseract-OCR'

In [3]:
class Item(BaseModel):
    url: str

In [2]:
app = FastAPI()

In [8]:
def getKeyWords(text):
    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 20

    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    
    return keywords

In [4]:
def getimgdescptext(filename):
    pdf_file = fitz.open(filename)
    pdf_images = []
    imgdescp=[]
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('textrank')
    for page_ndex in range(len(pdf_file)):
        page = pdf_file[page_ndex]
        image_list= pdf_file.get_page_images(page_ndex)
        if image_list:
            # tottext = page.extract_text()
            
            doc = nlp(page.extract_text())
            for sent in doc.sents:
                if 'figure' in sent.text.lower():
                    imgdescp.append(sent.text)
                if 'fig' in sent.text.lower():
                    imgdescp.append(sent.text)
                
    return imgdescp
                
            

In [15]:
def getImgDescp(filename):
    pdf_file = fitz.open(filename)
    pdf_image_list=[]
    imgdescps=[]

    for page_index in range(len(pdf_file)):
    
        page = pdf_file[page_index]
        image_list = pdf_file.get_page_images(page_index)
        if image_list:
            pdf_image_list.append(image_list)
            
        for image_index, img in enumerate(pdf_file.get_page_images(page), start=1):
            xref = img[0]
            pix = fitz.Pixmap(pdf_file,xref)
            piximg = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            # piximgarr = np.array(pix)
            decp=''
            decp=pytesseract.image_to_string(piximg)
            imgdescps.append(decp)
            
            
    return imgdescps

In [12]:
def getabstract(filereader):
    fulltext = []
    for i in filereader.pages:
        fulltext+= [i.extract_text()]
        currpage = i.extract_text().lower()
        if 'abstract' in currpage:
            abstract = currpage[currpage.find('abstract'):]
            # print('abstrat',abstract)
            return abstract
        if 'a b s t r a c t' in currpage:
            abstract = currpage[currpage.find('a b s t r a c t'):]
            # print('abstract',abstract)
            return abstract
    
    return ' '.join(fulltext[:4])

In [13]:
def getsummary(text):
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('textrank')
    doc = nlp(text)
    summ1 = list(doc._.textrank.summary())
    tostring = lambda sent:sent.text
    summary = list(map(tostring,summ1))
    summary = ' '.join(summary)
    return summary

In [16]:
def main(filename):
    text=''
    # filename=''
    file = open(filename,'rb')
    filereader = PyPDF2.PdfReader(file)
    text = getabstract(filereader)
    resp = {'summary':'','keywords':'',"img_descp":''}
    resp['keywords'] = getKeyWords(text)
    resp['summary']= getsummary(text)
    resp['img_descp'] = getImgDescp(filename)
    return resp

In [17]:
def test(url):
    #insert AI Model
    return url

In [5]:
@app.post('/nlpmodel/')
async def create_item(item: Item):
    return test(item.url)